In [428]:
import pandas as pd
from pandas.api.types import is_string_dtype
import numpy as np
import re
import plotly.express as px
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.options.display.max_colwidth = 1000

In [429]:
# Functions: 
def get_homogenization(table):
    # columns in lower case
    table.columns = table.columns.str.lower()
    table.columns = [re.sub('(,)|(:)|(-)', '', x) for x in table.columns]
    table.columns = [re.sub('(_)', ' ', x) for x in table.columns]
    table.columns = [x.strip(' ') for x in table.columns]

    # patient id column name normalization
    if 'subject number' in table.columns: 
        table_copy = table.rename(mapper={'subject number': 'patient id'}, axis=1)
    elif 'patient id' in table.columns: 
        table_copy = table.rename(mapper={'patient id': 'patient id'}, axis=1)
    
    # put string columns in lower case
    for col in table_copy.columns:
        if is_string_dtype(table_copy[col]):
            table_copy[col] = table_copy[col].str.lower()
            table_copy[col] = [re.sub('(,)|(:)|(-)', '', x) if isinstance(x, str) else x for x in table_copy[col]]
            table_copy[col] = [re.sub('(_)', ' ',x) if isinstance(x, str) else x for x in table_copy[col]]
            table_copy[col] = [x.strip(' ') if isinstance(x, str) else x for x in table_copy[col]]

    return table_copy


def get_unique_value_col(table, string=False):
    '''
    table: pandas dataframe
    string: computes unique value only for columns containing string
    ''' 
    unique_value = []
    unique_value_str = set()
    for col in table.columns:
        unique_value.append(list(pd.unique(table[col])))
        if string: 
            if is_string_dtype(table[col]):
                table[col] = table[col].str.lower()
                #get unique value for string col  
                if col != 'patient id':
                    unique_value_str.update(set(pd.unique(table[col])))
    if string: 
        #even when the column dtype is str, some values are not string
        unique_value_str_clean = unique_value_str.copy()
        for el in unique_value_str:
            if not(isinstance(el, str)):
                unique_value_str_clean.remove(el)
        return(unique_value_str_clean)
    
    return pd.DataFrame({"unique_value": unique_value}, index=table.columns)     


def check_merging(table1, table2, merged, columns_merged):
    for col in columns_merged:
        NA_table1 = table1[col].isna().sum()
        NA_table2 = table2[col].isna().sum()
        NA_merged = merged[col].isna().sum()
    if not(np.equal((NA_table1 + NA_table2),NA_merged)):
        return print(f"error in {col}")
    return print('ok')

def get_plot_na(table, axis):
    na_count = table.isna().sum(axis=axis).sort_values(ascending=False)
    if axis==0:
        labels={'index': 'number of missing value', 'value':'number of patient'}
        fig = px.bar(na_count, height=600, width=2000, labels=labels)

    else:
        labels={'index': 'features', 'value':'missing value count'}
        fig = px.bar(na_count.value_counts().sort_index(), height=600, width=2000, labels=labels)

    fig.update_layout(font_size=8, bargap=0.3)
    return fig


# Loading

In [430]:
filepath_stroke = "../data/cerebral_elderly_stroke/subjects.csv"
filepath_perfusion = '../data/cerebral_perfusion_data/data_description/GE-75_data_summary_table.csv'
filepath_diabetes = '../data/cerebral_elderly_diabetes/Data_Description/GE-79_Summary_Table-Demographics-MRI-Part1.csv'
filepath_desc = '../data/cerebral_perfusion_data/data_description/GE-75_data_dictionary.csv'
df_stroke = pd.read_csv(filepath_stroke)
df_perfusion = pd.read_csv(filepath_perfusion, header=0)
df_diabetes = pd.read_csv(filepath_diabetes)
df_desc = pd.read_csv(filepath_desc, usecols=range(5))

In [431]:
df_stroke.head()

,subject_number,completed_visit_status,htn_patient_medical_history,group,group2,age,height/m,mass/kg,bmi,gender,ethnicity,race,DM Non-DM STROKE,previous_tobacco_use,current_tobacco_use,pack_years,years,PREVIOUS ALCOHOL USE,ALCOHOL Dose/Week,Neuropathy AUTONOMIC SYMPTOMS,Dizziness AUTONOMIC SYMPTOMS,Numbness AUTONOMIC SYMPTOMS,Painful feet AUTONOMIC SYMPTOMS,Syncope AUTONOMIC SYMPTOMS,OH AUTONOMIC SYMPTOMS,"cancer,# FAMILY HISTORY",cancSpec FAMILY HISTORY,HeartDisease FAMILY HISTORY,Hdspecific FAMILY HISTORY,HTN FAMILY HISTORY,HTNspecific FAMILY HISTORY,DM FAMILY HISTORY,Dmspecific FAMILY HISTORY,StrokeFAMILY HISTORY,StrokeSpecific FAMILY HISTORY,HTN YRS PATIENT MEDICAL HISTORY,CANCER PATIENT MEDICAL HISTORY,STROKE PATIENT MEDICAL HISTORY,STROKE YR PATIENT MEDICAL HISTORY,ATRIAL FIBTRILLATION PATIENT MEDICAL HISTORY,HEART FAILURE =CHF /IFARCTION=-MI PATIENT MEDICAL HISTORY,HYPERLIPIDEMIA PATIENT MEDICAL HISTORY,DM PATIENT MEDICAL HISTORY,Stroke Date,NIHSS,MRS,Stroke Group,Stroke Side,Stroke Etiology,Symptoms,Lesions,Carotids,Tapered Medications taken during Visit 2,ANTIPLATELETS,ANTICOAGULANTS,ANTIHYPERLIPIDEMIC,ANTIPARKINSONIAN,STATINS,ESTROGEN,ACE INHIBITORS,ARBS,BETA BLOCKERS,DIURETICS,CA ++ BLOCKERS,INSULIN(Yes_or_No),ORAL HYPOGLYCEMIC,MRI YES OR NO,WBC K/uL,RBC m/uL,Hgb g/dL,Hgb<12.5,Hct %,MCV fL,MCH pg,MCHC %,RDW%,Plt ct K/uL,GLUCOSE mg/dL,CHOLESTmg/dL,TRIGLYCmg/dL,HDL mg/dL,CHOL/HD Ratio,LDL CALCmg/dL,SICAM (ng/ml),SVCAM (ng/ml),CRP (mg/L),IL-6 (pg/ml),TNFa (pg/ml),Erythropoietin(EPO),lnEPO,(Baseline Mean) HR BP BASELINE,SBP BASELINE,DBP BASELINE,MBP,SYST MCAR BASELINE,DIAST MCAR BASELINE,MEAN MCAR BASELINE,SYST MCAL BASELINE,DIAST MCAL BASELINE,MEAN MCAL BASELINE,SYST RA BASELINE,DIAST RA BASELINE,MEAN RA BASELINE,CO2 BASELINE,(Tilt mn) HR TILT,SBP TILT,DBP TILT,MBP TILT,SYST MCAR TILT,DIAST MCAR TILT,MEAN MCAR TILT,SYST MCAL TILT,DIAST MCAL TILT,MEAN MCAL TILT,SYSRA TILT,DIA RA TILT,MEAN RA TILT,CO2 TILT,(Hyper mn) HR HYPERVENTILATION,SBP HV,DBP HV,MBP HV,SYST MCAR HV,DIAST MCAR HV,MEAN MCAR HV,SYST MCAL HV,DIAST MCAL HV,MEAN MCAL HV,SYSRA HV,DIA RA HV,MEAN RA HV,CO2 HV,(Hypo mn) HR SUPINE REBREATHING,SBP SUPINE REBREATHING,DBP SUPINE REBREATHING,MBP SUPINE REBREATHING,SYST MCAR SUPINE REBREATHING,DIAST MCAR SUPINE REBREATHING,MEAN MCAR SUPINE REBREATHING,SYST MCAL SUPINE REBREATHING,DIAST MCAL SUPINE REBREATHING,MEAN MCAL SUPINE REBREATHING,SYSRA SUPINE REBREATHING,DIA RA SUPINE REBREATHING,MEAN RA SUPINE REBREATHING,CO2 SUPINE REBREATHING,(SitEO mn) SitEO HR mean,Systolic BP SitEO,Diastolic BP SitEO,Mean BP SitEO,Sys MCAR SitEO,DIa MCAR SitEO,Mean MCAR SitEO,Sys MCAL SitEO,Dia MCAL SitEO,Mean MCAL SitEO,Sys RAAR SitEO,Dia RAAR SitEO,Mean RAAR SitEO,ETCO2 SitEO,(StandEO mn) Mean HR StandEO,Sys BP StandEO,Dia BP StandEO,Mean BP StandEO,Sys MCAR StandEO,Dia MCAR StandEO,Mean MCAR StandEO,Sys MCAL StandEO,Dia MCAL StandEO,Mean MCAL StandEO,Sys RAAR StandEO,Dia RAAR StandEO,Mean RAAR StandEO,ETCO2 StandEO,(SitEC mn) Mean HR SitEC,Sys BP SitEC,Dia BP SitEC,Mean BP SitEC,Sys MCAR SitEC,Dia MCAR SitEC,Mean MCAR SitEC,Sys MCAL SitEC,Dia MCAL SitEC,Mean MCAL SitEC,Sys RAAR SitEC,Dia RAAR SitEC,Mean RAAR SitEC,ETCO2 SitEC,(StandEC mn) Mean HR StandEC,Sys BP StandEC,Dia BP StandEC,Mean BP StandEC,Sys MCAR StandEC,Dia MCAR StandEC,Mean MCAR StandEC,Sys MCAL StandEC,DIa MCAL StandEC,Mean MCAL StandEC,Sys RAAR StandEC,Dia RAAR StandEC,Mean RAAR StandEC,ETCO2 StandEC,Age_Residual MEAN MCAR BASELINE,Age_Residual MEAN MCAL BASELINE,Age_adjusted_Mean_MCAR_BASELINE,Age_adjusted_Mean_MCAL_BASELINE,Age_adjusted_Mean_MCARL_BASELINE,CO2_reactivity_MCAR,CO2_reactivity_MCAL,DELTA MEAN MCAR TILT-BASELINE,DELTA MEAN MCAL TILT-BASELINE,DELTA MEAN BP TILT-BASELINE,# Falls within last year,Falls-related injuries,24Hour-Daytime-SBP,24Hour-Nighttime-SBP,24Hour-Daytime-DBP,24Hour-Nighttime-DBP,24Hour-Daytime-MBP,24Hour-Nightime-MBP,24Hour-Daytime-HR,24Hour-Nightime-HR,24Hour-Daytime-PP,24Hour-Nightime-PP,24Hour-SBPDIP,24Hour-SBPDIP%,24Hour-DBPDIP,24H

In [432]:
df_diabetes.head()

,Patient ID,study,Visit,Group,Diabetes Duration,HTN or not,Race,"DM, Non-DM, STROKE",HEIGHT (M),MASS (KG),BMI,global GM vol,global GM vol covered in template,L superior frontal gyrus (#21) GM,R superior frontal gyrus (#22) GM,L middle frontal gyrus (#23) GM,R middle frontal gyrus (#24) GM,L inferior frontal gyrus (#25) GM,R inferior frontal gyrus (#26) GM,L precentral gyrus (#27) GM,R precentral gyrus (#28) GM,L middle orbitofrontal gyrus (#29) GM,R middle orbitofrontal gyrus (#30) GM,L lateral orbitofrontal gyrus (#31) GM,R lateral orbitofrontal gyrus (#32) GM,L gyrus rectus (#33) GM,R gyrus rectus (#34) GM,L postcentral gyrus (#41) GM,R postcentral gyrus (#42) GM,L superior parietal gyrus (#43) GM,R superior parietal gyrus (#44) GM,L supramarginal gyrus (#45) GM,R supramarginal gyurs (#46) GM,L angular gyrus (#47) GM,R angular gyrus (#48) GM,L precuneus (#49) GM,R precuneus (#50) GM,L superior occipital gyrus (#61) GM,R superior occipital gyrus (#62) GM,L middle occipital gyrus (#63) GM,R middle occipital gyrus (#64) GM,L inferior occipital gyrus (#65) GM,R inferior occipital gyrus (#66) GM,L cuneus (#67) GM,R cuneus (#68) GM,L superior temporal gyrus (#81) GM,R superior temporal gyrus (#82) GM,L middle temporal gyrus (#83) GM,R middle temporal gyrus (#84) GM,L inferior temporal gyrus (#85) GM,R inferior temporal gyrus (#86) GM,L parahippocampal gyrus (#87) GM,R parahippocampal gyrus (#88) GM,L lingual gyrus (#89) GM,R lingual gyrus (#90) GM,L fusiform gyrus (#91) GM,R fusiform gyrus (#92) GM,L insular cortex (#101) GM,R insular cortex (#102) GM,L cingulate gyrus (#121) GM,R cingulate gyrus (#122) GM,L caudate (#161) GM,R caudate (#162) GM,L putamen (#163) GM,R putamen (#164) GM,L hippocampus (#165) GM,R hippocampus (#166) GM,cerebellum (#181) GM,brain stem (#182) GM,global WM vol,global WM vol covered in template,L superior frontal gyrus (#21) WM,R superior frontal gyrus (#22) WM,L middle frontal gyrus (#23) WM,R middle frontal gyrus (#24) WM,L inferior frontal gyrus (#25) WM,R inferior frontal gyrus (#26) WM,L precentral gyrus (#27) WM,R precentral gyrus (#28) WM,L middle orbitofrontal gyrus (#29) WM,R middle orbitofrontal gyrus (#30) WM,L lateral orbitofrontal gyrus (#31) WM,R lateral orbitofrontal gyrus (#32) WM,L gyrus rectus (#33) WM,R gyrus rectus (#34) WM,L postcentral gyrus (#41) WM,R postcentral gyrus (#42) WM,L superior parietal gyrus (#43) WM,R superior parietal gyrus (#44) WM,L supramarginal gyrus (#45) WM,R supramarginal gyurs (#46) WM,L angular gyrus (#47) WM,R angular gyrus (#48) WM,L precuneus (#49) WM,R precuneus (#50) WM,L superior occipital gyrus (#61) WM,R superior occipital gyrus (#62) WM,L middle occipital gyrus (#63) WM,R middle occipital gyrus (#64) WM,L inferior occipital gyrus (#65) WM,R inferior occipital gyrus (#66) WM,L cuneus (#67) WM,R cuneus (#68) WM,L superior temporal gyrus (#81) WM,R superior temporal gyrus (#82) WM,L middle temporal gyrus (#83) WM,R middle temporal gyrus (#84) WM,L inferior temporal gyrus (#85) WM,R inferior temporal gyrus (#86) WM,L parahippocampal gyrus (#87) WM,R parahippocampal gyrus (#88) WM,L lingual gyrus (#89) WM,R lingual gyrus (#90) WM,L fusiform gyrus (#91) WM,R fusiform gyrus (#92) WM,L insular cortex (#101) WM,R insular cortex (#102) WM,L cingulate gyrus (#121) WM,R cingulate gyrus (#122) WM,L caudate (#161) WM,R caudate (#162) WM,L putamen (#163) WM,R putamen (#164) WM,L hippocampus (#165) WM,R hippocampus (#166) WM,cerebellum (#181) WM,brain stem (#182) WM,global CSF vol,global CSF vol covered in template,L superior frontal gyrus (#21) CSF,R superior frontal gyrus (#22) CSF,L middle frontal gyrus (#23) CSF,R middle frontal gyrus (#24) CSF,L inferior frontal gyrus (#25) CSF,R inferior frontal gyrus (#26) CSF,L precentral gyrus (#27) CSF,R precentral gyrus (#28) CSF,L middle orbitofrontal gyrus (#29) CSF,R middle orbitofrontal gyrus (#30) CSF,L lateral orbitofrontal gyrus (#31) CSF,R lateral orbitofrontal gyrus (#32) CSF,L gyrus rectus (#33) CSF,R gyrus rectu

In [433]:
df_perfusion.head()

,patient ID,study,Group,template,Diabetes Duration,HTN or not,HTN2,age,sex,Gender,HEIGHT (M),MASS (KG),BMI,global GM vol,global GM vol covered in template,L superior frontal gyrus (#21) GM,R superior frontal gyrus (#22) GM,L middle frontal gyrus (#23) GM,R middle frontal gyrus (#24) GM,L inferior frontal gyrus (#25) GM,R inferior frontal gyrus (#26) GM,L precentral gyrus (#27) GM,R precentral gyrus (#28) GM,L middle orbitofrontal gyrus (#29) GM,R middle orbitofrontal gyrus (#30) GM,L lateral orbitofrontal gyrus (#31) GM,R lateral orbitofrontal gyrus (#32) GM,L gyrus rectus (#33) GM,R gyrus rectus (#34) GM,L postcentral gyrus (#41) GM,R postcentral gyrus (#42) GM,L superior parietal gyrus (#43) GM,R superior parietal gyrus (#44) GM,L supramarginal gyrus (#45) GM,R supramarginal gyurs (#46) GM,L angular gyrus (#47) GM,R angular gyrus (#48) GM,L precuneus (#49) GM,R precuneus (#50) GM,L superior occipital gyrus (#61) GM,R superior occipital gyrus (#62) GM,L middle occipital gyrus (#63) GM,R middle occipital gyrus (#64) GM,L inferior occipital gyrus (#65) GM,R inferior occipital gyrus (#66) GM,L cuneus (#67) GM,R cuneus (#68) GM,L superior temporal gyrus (#81) GM,R superior temporal gyrus (#82) GM,L middle temporal gyrus (#83) GM,R middle temporal gyrus (#84) GM,L inferior temporal gyrus (#85) GM,R inferior temporal gyrus (#86) GM,L parahippocampal gyrus (#87) GM,R parahippocampal gyrus (#88) GM,L lingual gyrus (#89) GM,R lingual gyrus (#90) GM,L fusiform gyrus (#91) GM,R fusiform gyrus (#92) GM,L insular cortex (#101) GM,R insular cortex (#102) GM,L cingulate gyrus (#121) GM,R cingulate gyrus (#122) GM,L caudate (#161) GM,R caudate (#162) GM,L putamen (#163) GM,R putamen (#164) GM,L hippocampus (#165) GM,R hippocampus (#166) GM,cerebellum (#181) GM,brain stem (#182) GM,global WM vol,global WM vol covered in template,L superior frontal gyrus (#21) WM,R superior frontal gyrus (#22) WM,L middle frontal gyrus (#23) WM,R middle frontal gyrus (#24) WM,L inferior frontal gyrus (#25) WM,R inferior frontal gyrus (#26) WM,L precentral gyrus (#27) WM,R precentral gyrus (#28) WM,L middle orbitofrontal gyrus (#29) WM,R middle orbitofrontal gyrus (#30) WM,L lateral orbitofrontal gyrus (#31) WM,R lateral orbitofrontal gyrus (#32) WM,L gyrus rectus (#33) WM,R gyrus rectus (#34) WM,L postcentral gyrus (#41) WM,R postcentral gyrus (#42) WM,L superior parietal gyrus (#43) WM,R superior parietal gyrus (#44) WM,L supramarginal gyrus (#45) WM,R supramarginal gyurs (#46) WM,L angular gyrus (#47) WM,R angular gyrus (#48) WM,L precuneus (#49) WM,R precuneus (#50) WM,L superior occipital gyrus (#61) WM,R superior occipital gyrus (#62) WM,L middle occipital gyrus (#63) WM,R middle occipital gyrus (#64) WM,L inferior occipital gyrus (#65) WM,R inferior occipital gyrus (#66) WM,L cuneus (#67) WM,R cuneus (#68) WM,L superior temporal gyrus (#81) WM,R superior temporal gyrus (#82) WM,L middle temporal gyrus (#83) WM,R middle temporal gyrus (#84) WM,L inferior temporal gyrus (#85) WM,R inferior temporal gyrus (#86) WM,L parahippocampal gyrus (#87) WM,R parahippocampal gyrus (#88) WM,L lingual gyrus (#89) WM,R lingual gyrus (#90) WM,L fusiform gyrus (#91) WM,R fusiform gyrus (#92) WM,L insular cortex (#101) WM,R insular cortex (#102) WM,L cingulate gyrus (#121) WM,R cingulate gyrus (#122) WM,L caudate (#161) WM,R caudate (#162) WM,L putamen (#163) WM,R putamen (#164) WM,L hippocampus (#165) WM,R hippocampus (#166) WM,cerebellum (#181) WM,brain stem (#182) WM,global CSF vol,global CSF vol covered in template,L superior frontal gyrus (#21) CSF,R superior frontal gyrus (#22) CSF,L middle frontal gyrus (#23) CSF,R middle frontal gyrus (#24) CSF,L inferior frontal gyrus (#25) CSF,R inferior frontal gyrus (#26) CSF,L precentral gyrus (#27) CSF,R precentral gyrus (#28) CSF,L middle orbitofrontal gyrus (#29) CSF,R middle orbitofrontal gyrus (#30) CSF,L lateral orbitofrontal gyrus (#31) CSF,R lateral orbitofrontal gyrus (#32) CSF,L gyrus rectus (#33) CSF,R gyrus rectus (

# Exploration

Let's check if there are common patients between the datasets

In [434]:
set_diabetes = set(df_diabetes["Patient ID"].str.lower())
set_perfusion = set(df_perfusion["patient ID"].str.lower())
set_stroke = set(df_stroke["subject_number"].str.lower())

In [435]:
# Common elements
print(f'all three {len(set_diabetes & set_perfusion & set_stroke)}')
print(f'perfusion - diabetes {len(set_diabetes & set_perfusion)}')
print(f'perfusion - stroke {len(set_perfusion & set_stroke)}')
print(f'diabetes - stroke {len(set_diabetes & set_stroke)}')

all three 0
perfusion - diabetes 11
perfusion - stroke 1
diabetes - stroke 14


In [436]:
# Elements in
print(f'diabetes in stroke {(df_diabetes["Patient ID"].str.lower()).isin(set_stroke).sum()}')
print(f'diabetes in perfusion {(df_diabetes["Patient ID"].str.lower()).isin(set_perfusion).sum()}')

print(f'stroke in diabetes {(df_stroke["subject_number"].str.lower()).isin(set_diabetes).sum()}')
print(f'stroke in perfusion {(df_stroke["subject_number"].str.lower()).isin(set_perfusion).sum()}')

print(f'perfusion in diabetes {(df_perfusion["patient ID"].str.lower()).isin(set_diabetes).sum()}')
print(f'perfusion in stroke {(df_perfusion["patient ID"].str.lower()).isin(set_stroke).sum()}')

diabetes in stroke 24
diabetes in perfusion 18
stroke in diabetes 14
stroke in perfusion 1
perfusion in diabetes 11
perfusion in stroke 1


The diabetes dataset is missing important features (age, gender, ...).
Plus, stroke and perfusion datasets present in addition of the all the required demographic features, medical records and some lifestyle features: alcohol and tabacco use. 

Let's check the common patient of perfusion and stroke dataset

In [437]:
common_patient = set_perfusion & set_stroke

In [438]:
df_perfusion_homogenized = get_homogenization(df_perfusion)
df_stroke_homogenized =  get_homogenization(df_stroke)
print(df_perfusion_homogenized[df_perfusion_homogenized["patient id"].isin(common_patient)].isna().sum(axis=1)/df_perfusion_homogenized.shape[1])
print(df_stroke_homogenized[df_stroke_homogenized["patient id"].isin(common_patient)].isna().sum(axis=1)/df_stroke_homogenized.shape[1])

67    0.955896
dtype: float64
166    0.889391
dtype: float64


This patient has a lot of missing value and would be removed from the dataset

# Stroke & Perfusion datasets

## Loading v1

Stroke and perfusion datasets have been pre-filtered in different notebooks

In [439]:
filepath_stroke_v1 = "../data/cerebral_elderly_stroke/exploration/cerebral_elderly_stroke_v1.csv"
filepath_perfusion_v1 = "../data/cerebral_perfusion_data/exploration/cerebral_perfusion_diabetes_v1.csv"
df_stroke_v1 = pd.read_csv(filepath_stroke_v1)
df_perfusion_v1 = pd.read_csv(filepath_perfusion_v1)

In [440]:
df_stroke_v1.head()

,subject_number,completed_visit_status,htn_patient_medical_history,group,group2,age,height/m,mass/kg,bmi,gender,ethnicity,race,DM Non-DM STROKE,previous_tobacco_use,current_tobacco_use,pack_years,years,PREVIOUS ALCOHOL USE,ALCOHOL Dose/Week,Neuropathy AUTONOMIC SYMPTOMS,Dizziness AUTONOMIC SYMPTOMS,Numbness AUTONOMIC SYMPTOMS,Painful feet AUTONOMIC SYMPTOMS,Syncope AUTONOMIC SYMPTOMS,OH AUTONOMIC SYMPTOMS,"cancer,# FAMILY HISTORY",HeartDisease FAMILY HISTORY,HTN FAMILY HISTORY,DM FAMILY HISTORY,StrokeFAMILY HISTORY,CANCER PATIENT MEDICAL HISTORY,STROKE PATIENT MEDICAL HISTORY,ATRIAL FIBTRILLATION PATIENT MEDICAL HISTORY,HEART FAILURE =CHF /IFARCTION=-MI PATIENT MEDICAL HISTORY,HYPERLIPIDEMIA PATIENT MEDICAL HISTORY,DM PATIENT MEDICAL HISTORY,Stroke Group,ANTIPLATELETS,ANTICOAGULANTS,ANTIHYPERLIPIDEMIC,ANTIPARKINSONIAN,STATINS,ESTROGEN,ACE INHIBITORS,ARBS,BETA BLOCKERS,DIURETICS,CA ++ BLOCKERS,INSULIN(Yes_or_No),ORAL HYPOGLYCEMIC,WBC K/uL,RBC m/uL,Hgb g/dL,Hgb<12.5,Hct %,MCV fL,MCH pg,MCHC %,RDW%,Plt ct K/uL,GLUCOSE mg/dL,CHOLESTmg/dL,TRIGLYCmg/dL,HDL mg/dL,CHOL/HD Ratio,LDL CALCmg/dL,CRP (mg/L)
0,S0011,INELIGIBLE,YES,CONTROL,CONTROL-HTN,70,1.6383,71.667594,26.701519,F,Non-H/L,White,Non-DM,YES,NO,20.0,40.0,YES,0.0,NO,NO,NO,NO,NO,NO,0,1,0,0,0,NO,NO,NO,NO,NO,NO,Non-Stroke,N,N,N,N,N,N,N,Y,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.67
1,S0030,COMPLETED,YES,CONTROL,CONTROL-NTN,64,1.6256,72.574779,27.463655,F,Non-H/L,White,Non-DM,NO,NO,0.0,0.0,YES,0.0,NO,NO,NO,NO,NO,NO,0,0,0,0,0,NO,NO,NO,NO,NO,NO,Non-Stroke,N,N,N,N,N,N,N,N,N,N,N,N,N,5.7,4.65,14.1,Non-anemic,41.7,90.0,30.2,33.8,12.9,252.0,74.0,270.0,242.0,66.0,4.1,156.0,1.99
2,S0033,EXCLUDED,no,CONTROL,CONTROL-NTN,65,1.8200,92.986000,28.072093,M,Non H/L,WHITE,Non-DM,YES,NO,10.0,10.0,yes,1.0,no,NO,NO,NO,NO,NO,0,2,1,1,1,no,no,NO,No,no,no,Non-Stroke,n,n,N,N,N,n,Y,N,N,N,N,N,N,7.2,4.40,14.3,Non-anemic,39.3,89.0,32.5,36.4,14.3,206.0,76.0,160.0,98.0,44.0,3.6,96.0,0.32
3,S0044,COMPLETED,NO,CONTROL,CONTROL-HTN,74,1.7018,68.038855,23.493080,F,Non-H/L,White,Non-DM,YES,NaN,40.0,40.0,YES,2.0,NO,YES,YES,NO,NO,YES,3,0,0,0,0,NO,NO,NO,NO,NO,NO,Non-Stroke,N,N,N,N,N,N,N,N,N,N,N,N,N,6.1,4.54,15.0,Non-anemic,44.0,97.0,33.0,34.1,13.6,242.0,66.0,189.0,139.0,60.0,3.2,101.0,3.70
4,S0064,COMPLETED,YES,CONTROL,CONTROL-NTN,76,1.7018,68.038855,23.493080,M,Non-H/L,White,Non-DM,NO,NO,0.0,0.0,YES,0.0,NO,NO,NO,NO,NO,NO,0,2,0,0,0,NO,NO,NO,NO,YES,NO,Non-Stroke,N,N,N,N,Y,N,N,N,N,N,N,N,N,5.7,4.48,14.9,Non-anemic,43.6,97.0,33.1,34.1,12.3,180.0,75.0,204.0,136.0,44.0,4.6,133.0,1.82


In [441]:
df_perfusion_v1.head()

,patient ID,study,Group,Diabetes Duration,age,sex,HEIGHT (M),MASS (KG),BMI,Ethnicity,"DM, Non-DM, STROKE",PREVIOUS TOBACCO USE,CURRENT TOBACCO USE,PackYears,Years,PREVIOUS ALCOHOL USE,ALCOHOL Dose/Week,Neuropathy AUTONOMIC SYMPTOMS,Dizziness AUTONOMIC SYMPTOMS,Numbness AUTONOMIC SYMPTOMS,Painful feet AUTONOMIC SYMPTOMS,Syncope AUTONOMIC SYMPTOMS,OH AUTONOMIC SYMPTOMS,"cancer,# FAMILY HISTORY",HeartDisease FAMILY HISTORY,HTN FAMILY HISTORY,DM FAMILY HISTORY,Dmspecific FAMILY HISTORY,StrokeFAMILY HISTORY,CANCER PATIENT MEDICAL HISTORY,STROKE PATIENT MEDICAL HISTORY,ATRIAL FIBTRILLATION PATIENT MEDICAL HISTORY,HEART FAILURE =CHF /IFARCTION=-MI PATIENT MEDICAL HISTORY,HYPERLIPIDEMIA PATIENT MEDICAL HISTORY,DM PATIENT MEDICAL HISTORY,Medications,ANTIPLATELETS,ANTICOAGULANTS,ANTIHYPERLIPIDEMIC,ANTIPARKINSONIAN,STATINS,ESTROGEN,ACE INHIBITORS,ARBS,BETA BLOCKERS,DIURETICS,CA ++ BLOCKERS,INSULIN(Yes_or_No),ORAL HYPOGLYCEMIC,OTHER 2,WBC K/uL,Hgb g/dL,Hgb<12.5,Hct %,MCV fL,MCH pg,MCHC %,Plt ct K/uL,GLUCOSE mg/dL,CHOLESTmg/dL,TRIGLYCmg/dL,HDL mg/dL,CHOL/HD Ratio,URINE CREAT mg/dL,URINE ALBUMIN mg/dL,Alb/Cre mg/g (URINE),Hb A1C%,CRP (mg/L)
0,S0250,GE75,DM,7.0,50,Male,1.790,114.65,35.782279,H/L,DM,YES,NO,2.86,10.0,YES,7.0,NO,NO,NO,NO,NO,NO,1,1,1,1,m,0,NO,NO,NO,NO,NO,YES,"glyburide, metformin hydrochloride, multivitamin",N,N,N,N,N,N,N,N,N,N,N,N,Y,multivitamin,6.9,15.0,Non-anemic,42.2,80.0,28.2,35.5,250.0,211.0,135.0,267.0,36.0,4.5,65.0,2.4,3.69,9.4,0.240
1,S0254,GE75,CONTROL,0.0,69,Female,1.531,56.90,24.275148,Non-H/L,Non-DM,YES,NO,0.14,1.0,YES,0.5,NO,NO,NO,NO,YES,NO,1,1,1,0,NaN,1,NO,NO,NO,NO,NO,NO,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,6.7,14.2,Non-anemic,39.1,87.0,31.7,36.5,242.0,95.0,176.0,95.0,59.0,3.0,137.0,8.0,58.40,5.1,0.104
2,S0255,GE75,DM,3.0,78,Female,1.520,65.77,28.466932,Non-H/L,DM,NO,No,0.00,0.0,Yes,9.0,No,No,No,NO,NO,NO,0,3,0,3,"f,b,si",0,No,No,NO,NO,YES,YES,"Ranatidine, Zocor, Inderal, Calcuim + Vitamin D, Metamucil",N,N,N,N,Y,N,N,N,Y,N,N,N,N,"Ranitidine, Calcium + D, Metamucil",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S0256,GE75,DM,23.0,65,Female,1.575,73.50,29.629630,Non-H/L,DM,YES,NO,5.71,20.0,YES,3.0,NO,NO,NO,NO,NaN,NO,2,1,0,3,"f,m,b",1,YES,NO,NO,NO,YES,YES,"lipitor, actos, effexor, synthroid, aspirin, evista, multivitamin, iron, calcium",Y,N,N,N,Y,N,N,N,N,N,N,N,Y,"effexor, synthroid, evista, multivitamin, iron, calcium",4.4,10.3,Anemic,31.3,87.0,28.6,32.8,315.0,111.0,194.0,111.0,75.0,2.6,69.0,0.3,4.30,7.1,0.060
4,s0257,GE75,CONTROL,0.0,55,Male,1.700,70.31,24.328720,Non H/L,Non-DM,NO,No,0.00,0.0,No,0.0,No,No,No,NO,NO,NO,0,1,1,0,NaN,1,No,No,NO,NO,NO,NO,Multi Vitamin,N,N,N,N,N,N,N,N,N,N,N,N,N,Multivitamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [442]:
df_perfusion_v1.shape

(88, 68)

In [443]:
df_stroke_v1.shape

(168, 67)

## First homogenization

In [444]:
df_stroke_v2 = get_homogenization(df_stroke_v1)
df_perfusion_v2 = get_homogenization(df_perfusion_v1) 
df_desc_v1 = get_homogenization(df_desc)

# Normalization of 'patient id' column name
print(df_perfusion_v2['patient id'].head())
print(df_stroke_v2['patient id'].head())
print(list(df_desc_v1["patient id"])[0:10])

# Adding a tag to distinguish each dataset  
df_perfusion_v2["study"] = "cerebral_perfusion_diabetes"
df_stroke_v2["study"] = "cerebral_elderly_stroke"

print(df_perfusion_v2.columns)
print(df_stroke_v2.columns)
print(df_perfusion_v2["study"].head())
print(df_stroke_v2["study"].head())

0    s0250
1    s0254
2    s0255
3    s0256
4    s0257
Name: patient id, dtype: object
0    s0011
1    s0030
2    s0033
3    s0044
4    s0064
Name: patient id, dtype: object
[nan, 'study', 'completed visits status', 'htn patient medical history', 'group 2', 'age', 'height (m)', 'mass (kg)', 'bmi', 'gender']
Index(['patient id', 'study', 'group', 'diabetes duration', 'age', 'sex',
       'height (m)', 'mass (kg)', 'bmi', 'ethnicity', 'dm nondm stroke',
       'previous tobacco use', 'current tobacco use', 'packyears', 'years',
       'previous alcohol use', 'alcohol dose/week',
       'neuropathy autonomic symptoms', 'dizziness autonomic symptoms',
       'numbness autonomic symptoms', 'painful feet autonomic symptoms',
       'syncope autonomic symptoms', 'oh autonomic symptoms',
       'cancer# family history', 'heartdisease family history',
       'htn family history', 'dm family history', 'dmspecific family history',
       'strokefamily history', 'cancer patient medical history',
 

### Common elements

In [445]:
common_col = set(df_perfusion_v2.columns) & set(df_stroke_v2.columns)
print(common_col)
print(len(common_col))

{'dm family history', 'heart failure =chf /ifarction=mi patient medical history', 'hgb g/dl', 'alcohol dose/week', 'wbc k/ul', 'estrogen', 'mchc %', 'dm nondm stroke', 'insulin(yes or no)', 'patient id', 'heartdisease family history', 'ace inhibitors', 'strokefamily history', 'dm patient medical history', 'stroke patient medical history', 'cancer patient medical history', 'atrial fibtrillation patient medical history', 'antiparkinsonian', 'years', 'previous tobacco use', 'mch pg', 'bmi', 'hct %', 'statins', 'syncope autonomic symptoms', 'glucose mg/dl', 'hgb<12.5', 'anticoagulants', 'hyperlipidemia patient medical history', 'chol/hd ratio', 'neuropathy autonomic symptoms', 'study', 'oral hypoglycemic', 'age', 'ethnicity', 'current tobacco use', 'antiplatelets', 'previous alcohol use', 'cancer# family history', 'beta blockers', 'dizziness autonomic symptoms', 'diuretics', 'ca ++ blockers', 'group', 'oh autonomic symptoms', 'mcv fl', 'triglycmg/dl', 'hdl mg/dl', 'cholestmg/dl', 'numbness

On 67 and respectively 68 features, the two datasets share 56 features (2 being the patient_id and the study features). Let's check the matches

In [446]:
desc_common_el = df_desc_v1[df_desc_v1["patient id"].isin(common_col)][["patient id", "variable description"]]
desc_common_el.set_index("patient id", inplace=True)

In [447]:
unique_value_stroke = []
unique_value_perfusion = []
symmetric_difference = []

for col in common_col:
    unique_stroke = list(pd.unique(df_stroke_v2[col]))
    unique_perfusion = list(pd.unique(df_perfusion_v2[col]))
    unique_value_stroke.append(unique_stroke)
    unique_value_perfusion.append(unique_perfusion)
    symmetric_difference.append((set(unique_stroke).symmetric_difference(set(unique_perfusion))))
          
df_common_features_comparison = pd.DataFrame({'stroke':unique_value_stroke, 'perfusion':unique_value_perfusion, "symmetric_diff": symmetric_difference}, index=common_col)
df_common_features_comparison["desc"] = desc_common_el
df_common_features_comparison

,stroke,perfusion,symmetric_diff,desc
dm family history,"[0, 1, nan, 2, 3, 4, unknown]","[1, 0, 3, 2, 4, nan, unknown]",{},diabetes mellitus family history
heart failure =chf /ifarction=mi patient medical history,"[no, nan, mi, ye]","[no, ch, ye, nan]","{ch, mi}",patient with heart failure/infarction history y/n
hgb g/dl,"[nan, 14.1, 14.3, 15.0, 14.9, 13.0, 15.2, 14.7, 16.9, 13.3, 14.5, 15.1, 12.4, 12.5, 11.7, 13.8, 12.8, 13.2, 14.8, 15.3, 12.3, 12.0, 13.5, 15.6, 13.1, 15.4, 14.0, 10.8, 12.1, 14.6, 16.3, 15.7, 16.1, 10.0, 13.6, 12.7, 15.9, 17.2, 14.4, 16.5, 11.9, 13.4, 15.8, 12.2, 16.6, 15.5, 16.4, 13.7, 11.3, 16.0]","[15.0, 14.2, nan, 10.3, 13.3, 12.1, 12.2, 12.9, 12.5, 16.2, 13.6, 10.6, 13.4, 12.0, 12.6, 14.0, 13.7, 16.8, 16.0, 14.8, 15.1, 12.4, 11.4, 14.6, 15.5, 12.7, 14.1, 15.9, 14.4, 14.9, 16.4, 11.6, 15.4, 15.8, 14.5, 13.9, 14.3, 13.0, 13.8, 12.3, 13.1, 14.7]","{nan, nan, 10.3, 10.6, 14.2, 12.9, 16.2, 16.8, 17.2, 12.8, 15.7, 10.8, 10.0, 11.4, 11.6, 11.7, 11.9, 11.3, 12.6, 13.9, 13.2, 15.2, 16.9, 16.3, 15.6, 16.1, 16.5, 16.6, 15.3, 13.5}",hemoglobin
alcohol dose/week,"[0.0, 1.0, 2.0, 7.0, 4.0, 3.0, 5.0, nan, 15.0, 12.0, 20.0, 70.0, 10.0, 1.5, 2.5, 28.0, 35.0, 21.0, 42.0, 6.0, 10.5, 49.0, 0.5, 0.25, 8.0, 3.5, 14.0, 24.0]","[7.0, 0.5, 9.0, 3.0, 0.0, nan, 0.25, 1.0, 70.0, 4.0, 14.0, 2.0, 35.0, 0.86, 21.0, 25.0, 5.0, 10.0]","{nan, 9.0, nan, 1.5, 2.5, 12.0, 10.5, 15.0, 3.5, 20.0, 24.0, 25.0, 28.0, 6.0, 0.86, 8.0, 42.0, 49.0}",alcohol dose/week
wbc k/ul,"[nan, 5.7, 7.2, 6.1, 5.5, 11.4, 5.8, 8.1, 7.0, 5.0, 6.5, 5.3, 6.9, 6.4, 7.8, 4.8, 3.9, 6.0, 6.7, 7.1, 5.1, 5.9, 8.0, 7.7, 9.6, 9.0, 8.7, 6.3, 8.3, 9.4, 3.8, 2.5, 5.4, 4.9, 4.5, 6.2, 8.8, 8.2, 9.2, 8.6, 5.2, 6.8, 9.9, 8.5, 2.9, 11.8, 12.8, 6.6, 4.6, 7.4, 10.2, 3.2, 11.1, 10.0, 10.3, 9.7, 8.9, 4.7, 13.2, 7.5, 8.4, 7.3]","[6.9, 6.7, nan, 4.4, 9.1, 6.1, 9.2, 4.9, 6.6, 8.5, 8.6, 5.6, 5.5, 5.9, 9.5, 5.2, 4.7, 8.0, 9.9, 6.2, 10.6, 10.4, 5.7, 7.3, 4.0, 7.2, 8.4, 8.3, 4.2, 8.7, 7.1, 4.8, 5.8, 6.4, 4.5, 3.6, 8.2, 6.8, 3.7, 7.0, 4.6, 7.5, 5.3, 6.3, 7.8, 11.3, 4.1, 5.4, 8.9]","{nan, nan, 2.5, 3.6, 4.4, 9.1, 5.6, 9.5, 10.6, 10.4, 11.3, 4.0, 4.1, 5.0, 5.1, 6.5, 6.0, 7.4, 8.1, 9.6, 8.8, 9.0, 9.4, 10.2, 10.0, 10.3, 9.7, 11.4, 11.8, 11.1, 12.8, 13.2, 3.7, 3.8, 3.2, 4.2, 7.7, 2.9, 3.9}",white blood cells count
estrogen,"[n, y, nan]","[n, y, nan]",{},medication classess
mchc %,"[nan, 33.8, 36.4, 34.1, 33.7, 33.9, 34.5, 36.6, 33.3, 34.2, 34.4, 34.7, 34.0, 35.0, 32.4, 34.6, 33.1, 33.4, 32.9, 33.0, 32.5, 35.3, 35.8, 35.9, 32.2, 33.6, 31.6, 33.5, 32.7, 34.3, 32.6, 35.5, 31.7, 34.9, 35.1, 35.7, 32.3, 32.8, 36.2, 35.4, 36.1, 34.8, 35.2, 32.0, 31.8]","[35.5, 36.5, nan, 32.8, 34.9, 33.7, 34.4, 35.4, 34.7, 33.6, 32.5, 33.0, 32.9, 32.7, 33.8, 33.9, 33.5, 34.0, 34.8, 33.1, 35.7, 34.2, 30.6, 33.2, 34.6, 32.2, 34.1, 35.2, 32.0, 34.3, 31.4, 31.6, 32.3, 31.5, 35.3, 32.4, 34.5, 35.6]","{nan, nan, 31.7, 31.5, 30.6, 31.4, 36.5, 35.6, 36.4, 36.6, 35.0, 36.2, 36.1, 35.1, 35.8, 35.9, 33.2, 33.3, 33.4, 31.8, 32.6}",mean corpuscular hemoglobin concentration
dm nondm stroke,"[nondm, stroke]","[dm, nondm]","{dm, stroke}",NaN
insulin(yes or no),"[n, nan]","[n, y, nan]",{y},medication classess
patient id,"[s0011, s0030, s0033, s0044, s0064, s0067, s0068, s0078, s0098, s0121, s0132, s0147, s0151, s0153, s0154, s0155, s0157, s0160, s0162, s0163, s0164, s0165, s0166, s0169, s0172, s0174, s0175, s0176, s0177, s0178, s0181, s0182, s0183, s0184, s0185, s0186, s0187, s0188, s0194, s0197, s0199, s0200, s0203, s0204, s0205, s0206, s0207, s0208, s0209, s0210, s0211, s0212, s0213, s0214, s0215, s0216, s0217, s0218, s0219, s0220, s0221, s0222, s0223, s0224, s0225, s0226, s0227, s0228, s0229, s0230, s0231, s0232, s0233, s0234, s0235, s0236, s0237, s0238, s0239, s0240, s0241, s0242, s0243, s0244, s0245, s0246, s0247, s0248, s0249, s0258, s0261, s0266, s0275, s0277, s0284, s0285, s0289, s0293, s0295, s0298, ...]","[s0250, s0254, s0255, s0256, s0257, s0262, s0264, s0267, s0270, s0271, s0273, s0274, s0279, s0281, s0282, s0283, s0287, s0288

Checking similarity
- atrial fibtrillation patient medical history: what is the /h ??
- insulin(yes_or_no): no 'yes' in the stroke dataset
- heart failure =chf /ifarction=-mi patient medical history
- numbness autonomic symptoms: '20' in stroke dataset
- group: DM/STROKE vs control
- stroke patient medical history: TIA in perfusion

In [448]:
df_perfusion_v2[df_perfusion_v2["atrial fibtrillation patient medical history"]=='h/']

,patient id,study,group,diabetes duration,age,sex,height (m),mass (kg),bmi,ethnicity,dm nondm stroke,previous tobacco use,current tobacco use,packyears,years,previous alcohol use,alcohol dose/week,neuropathy autonomic symptoms,dizziness autonomic symptoms,numbness autonomic symptoms,painful feet autonomic symptoms,syncope autonomic symptoms,oh autonomic symptoms,cancer# family history,heartdisease family history,htn family history,dm family history,dmspecific family history,strokefamily history,cancer patient medical history,stroke patient medical history,atrial fibtrillation patient medical history,heart failure =chf /ifarction=mi patient medical history,hyperlipidemia patient medical history,dm patient medical history,medications,antiplatelets,anticoagulants,antihyperlipidemic,antiparkinsonian,statins,estrogen,ace inhibitors,arbs,beta blockers,diuretics,ca ++ blockers,insulin(yes or no),oral hypoglycemic,other 2,wbc k/ul,hgb g/dl,hgb<12.5,hct %,mcv fl,mch pg,mchc %,plt ct k/ul,glucose mg/dl,cholestmg/dl,triglycmg/dl,hdl mg/dl,chol/hd ratio,urine creat mg/dl,urine albumin mg/dl,alb/cre mg/g (urine),hb a1c%,crp (mg/l)
37,s0323,cerebral_perfusion_diabetes,control,0.0,78,male,1.73,85.0,28.400548,nonh/l,nondm,no,no,0.0,0.0,yes,1.0,no,no,no,no,no,no,1,1,0,0,NaN,0,no,no,h/,no,yes,no,quinidine colace prostate medications vitamins,n,n,y,n,y,n,n,n,n,n,n,n,n,colace vitamin quinidine,8.7,14.6,nonanemic,42.5,90.0,30.8,34.4,345.0,90.0,165.0,94.0,48.0,3.4,79.0,0.8,10.1,5.9,0.9


### Common elements not detected

In [449]:
undetected_perfusion = (set(df_perfusion_v2.columns).difference(set(df_stroke_v2.columns)))
undetected_perfusion

{'alb/cre mg/g (urine)',
 'diabetes duration',
 'dmspecific family history',
 'hb a1c%',
 'height (m)',
 'mass (kg)',
 'medications',
 'other 2',
 'packyears',
 'sex',
 'urine albumin mg/dl',
 'urine creat mg/dl'}

In [450]:
undetected_stroke = set(df_stroke_v2.columns).difference(set(df_perfusion_v2.columns))
undetected_stroke

{'completed visit status',
 'gender',
 'group2',
 'height/m',
 'htn patient medical history',
 'ldl calcmg/dl',
 'mass/kg',
 'pack years',
 'race',
 'rbc m/ul',
 'rdw%',
 'stroke group'}

In [451]:
len(set(df_perfusion_v2.columns).difference(set(df_stroke_v2.columns))) + len(set(df_stroke_v2.columns).difference(set(df_perfusion_v2.columns)))

24

In [452]:
common_col

{'ace inhibitors',
 'age',
 'alcohol dose/week',
 'anticoagulants',
 'antihyperlipidemic',
 'antiparkinsonian',
 'antiplatelets',
 'arbs',
 'atrial fibtrillation patient medical history',
 'beta blockers',
 'bmi',
 'ca ++ blockers',
 'cancer patient medical history',
 'cancer# family history',
 'chol/hd ratio',
 'cholestmg/dl',
 'crp (mg/l)',
 'current tobacco use',
 'diuretics',
 'dizziness autonomic symptoms',
 'dm family history',
 'dm nondm stroke',
 'dm patient medical history',
 'estrogen',
 'ethnicity',
 'glucose mg/dl',
 'group',
 'hct %',
 'hdl mg/dl',
 'heart failure =chf /ifarction=mi patient medical history',
 'heartdisease family history',
 'hgb g/dl',
 'hgb<12.5',
 'htn family history',
 'hyperlipidemia patient medical history',
 'insulin(yes or no)',
 'mch pg',
 'mchc %',
 'mcv fl',
 'neuropathy autonomic symptoms',
 'numbness autonomic symptoms',
 'oh autonomic symptoms',
 'oral hypoglycemic',
 'painful feet autonomic symptoms',
 'patient id',
 'plt ct k/ul',
 'previous

In [453]:
common_col_undetected = [
('height (m)', 'height/m'), 
('mass (kg)', 'mass/kg'), 
('packyears', 'pack years'),
('sex', 'gender')
]

In [454]:
len(common_col_undetected)

4

### Uncommon features:

In [455]:
uncommon_stroke = ['completed_visit_status',
                   'group2',
                   'htn_patient_medical_history',
                   'ldl calcmg/dl',
                   'race',
                   'rbc m/ul',
                   'rdw%',
                   'stroke group']
uncommon_perfusion = [ 'alb/cre mg/g (urine)',
                       'diabetes duration', 
                       'dmspecific family history',
                       'hb a1c%',
                       'medications',
                       'other 2',
                       'study',
                       'urine albumin mg/d',
                       'urine creat mg/dl']

Bio measures can be excluded: 
- stroke:    
    - 'rbc m/ul'
    - 'rdw%'
    - 'ldl calcmg/dl'
- perfusion: 
    - 'alb/cre mg/g (urine)'
    - 'hb a1c%'
    - 'urine albumin mg/d'
    - 'urine creat mg/dl'

In [456]:
print(uncommon_stroke)
print(uncommon_perfusion)

['completed_visit_status', 'group2', 'htn_patient_medical_history', 'ldl calcmg/dl', 'race', 'rbc m/ul', 'rdw%', 'stroke group']
['alb/cre mg/g (urine)', 'diabetes duration', 'dmspecific family history', 'hb a1c%', 'medications', 'other 2', 'study', 'urine albumin mg/d', 'urine creat mg/dl']


In [457]:
print(pd.unique(df_stroke_v1['group2']))


['CONTROL-HTN' 'CONTROL-NTN' 'STROKE-HTN' 'STROKE']


In [458]:
print(pd.unique(df_stroke_v2["group2"]))
print(pd.unique(df_stroke_v2['htn patient medical history']))
print(pd.unique(df_stroke_v2["stroke group"]))
print(pd.unique(df_stroke_v2["stroke patient medical history"]))
print(pd.unique(df_perfusion_v2["stroke patient medical history"]))

['controlhtn' 'controlntn' 'strokehtn' 'stroke']
['yes' 'no' nan]
['nonstroke' 'stroke' 'tia' nan]
['no' 'yes' nan]
['no' 'tia']


- no info on hypertension in perfusion dataset: exclusion of hypertension
- different level of precision: stroke and TIA (mini stroke)

In [459]:
print(pd.unique(df_stroke_v2["race"]))
print(pd.unique(df_stroke_v2["ethnicity"]))
print(pd.unique(df_perfusion_v2["ethnicity"]))

['white' 'african american' 'asian' 'unknown' 'american indian' 'aa']
['nonh/l' 'non h/l' 'unknown' 'h/l']
['h/l' 'nonh/l' 'non h/l']


"race" & ethnicity are americain centered. Should discard these

In [460]:
print(pd.unique(df_perfusion_v2['diabetes duration'])) 
print(pd.unique(df_perfusion_v2['dmspecific family history'])) 
print(pd.unique(df_perfusion_v2['dm family history'])) 

[ 7.  0.  3. 23.  1. 26. 10. 16. nan  4. 18.  2. 15.  5.  9. 21.  6. 12.
 17. 37. 28. 25.  8. 32. 31. 20.]
['m' nan 'fbsi' 'fmb' 'gpmb' 'gp' 'mb' 'f' 'gpf' 'si' 'gpfmsi' 'gp f m'
 'gps' 'gpm' 'gpb' 'gpbsi' 'fm' 'b' 'fsi' 'm f' 'db' 'd' 'fd' 'gpfm' 'bsi'
 'gp m' 'gp si' 'gp  m si']
['1' '0' '3' '2' '4' nan 'unknown']


- diabetes duration has no equivalent 
- 'dmspecific family history' is redundant with the shared feature 'dm family history'

In [461]:
print(pd.unique(df_perfusion_v2['medications'])) 
print(len(pd.unique(df_perfusion_v2['medications'])))

['glyburide metformin hydrochloride multivitamin' nan
 'ranatidine zocor inderal calcuim + vitamin d metamucil'
 'lipitor actos effexor synthroid aspirin evista multivitamin iron calcium'
 'multi vitamin'
 'gabapentin diovan diltiazam protonix ranitidine fosamax lipitor flonase celebrex'
 'norvasc lexapro furosemide lipitor minocycline gabapentin lisinopril novolog naproxen aspirin nasal spray tums multivitamins propramaled'
 'toporol mavik glucotrol xl lipitor aspirin vitamin c multi vitamin'
 'coumadin metoprolol lisinopril furosemide pot chlor lovastatin fish oil gemfibrozil flomax omeprazole mirapex flonase astelin nasal spray metrocream tetracycline citrical + d cosamine ds folic acid vitamin c multi vitamin'
 'none'
 'aspirin atenolol bupropion fluoxetine glyburide losartan metformin simvastatin verapamil loratadine'
 'metformin lantus laxol aspirin humalin'
 'bupropion trazadone clonazepam glucophage'
 'zocor glucotrol metformin aspirin'
 'lipitor multivitamin magnesium potassiu

In [462]:
print(pd.unique(df_perfusion_v2['other 2'])) 
print(len(pd.unique(df_perfusion_v2['other 2']))) 

['multivitamin' nan 'ranitidine calcium + d metamucil'
 'effexor synthroid evista multivitamin iron calcium'
 'fosamax (ca+ regulator) celebrex (analgesic) flonase ranitidine gabapentin'
 'lexapro minocycline gabapentin naproxen' 'multivitamin vit c'
 'omeprazole astelin nasal spray metrocream tetracycline citrical + d cosamine ds folic acid vitamin c multi vitamin fish oil flonase'
 'bupropion fluoxetine loratadine' 'levoxyl'
 'buproprion trazadone clonazepam'
 'multivitam magnesium potassium calcium coenzyme q10 prilosec'
 'calcium multivitamin' 'provigil flexeril' 'methilmazole protonix'
 'levothyroxyl lorazepam citalopram glycolax powder prilosec otc magnesium calcium'
 'abilify celexa lorazepam trazodone allopurinol multivitamin' 'salt'
 'omega3 viamins' 'folic acid vitamin b12 allopurinol digitek tylenol'
 'fosamax niaspan ocuvite centrum silver' 'prilosec'
 'levothyroxine calcium fluoxetine' 'flonase' 'alupurinol'
 'ambien/lunesta (prn)' 'colace vitamin quinidine' 'levoxyl gabap

In [463]:
len(set(pd.unique(df_perfusion_v2['other 2'])) & (set(pd.unique(df_perfusion_v2['medications']))))

1

other 2 and medications represent different drugs taken by patients. 
Drug names are the commercial name

In [464]:
list(pd.unique(df_stroke_v2["completed visit status"]))

['ineligible',
 'completed',
 'excluded',
 'v1',
 'lost to followup',
 'excluded pending pcp']

In [465]:
list(pd.unique(df_perfusion_v2["study"])) # should keep a study tag

['cerebral_perfusion_diabetes']

In [466]:
df_stroke_v2[df_stroke_v2['completed visit status']=="v1"]

,patient id,completed visit status,htn patient medical history,group,group2,age,height/m,mass/kg,bmi,gender,ethnicity,race,dm nondm stroke,previous tobacco use,current tobacco use,pack years,years,previous alcohol use,alcohol dose/week,neuropathy autonomic symptoms,dizziness autonomic symptoms,numbness autonomic symptoms,painful feet autonomic symptoms,syncope autonomic symptoms,oh autonomic symptoms,cancer# family history,heartdisease family history,htn family history,dm family history,strokefamily history,cancer patient medical history,stroke patient medical history,atrial fibtrillation patient medical history,heart failure =chf /ifarction=mi patient medical history,hyperlipidemia patient medical history,dm patient medical history,stroke group,antiplatelets,anticoagulants,antihyperlipidemic,antiparkinsonian,statins,estrogen,ace inhibitors,arbs,beta blockers,diuretics,ca ++ blockers,insulin(yes or no),oral hypoglycemic,wbc k/ul,rbc m/ul,hgb g/dl,hgb<12.5,hct %,mcv fl,mch pg,mchc %,rdw%,plt ct k/ul,glucose mg/dl,cholestmg/dl,triglycmg/dl,hdl mg/dl,chol/hd ratio,ldl calcmg/dl,crp (mg/l),study
8,s0098,v1,no,control,controlntn,70,1.6764,71.667594,25.501606,m,nonh/l,white,nondm,no,NaN,0.0,0.0,yes,4.0,no,no,no,no,no,no,1,0,0,0,0,no,no,no,no,yes,no,nonstroke,n,n,n,n,n,n,n,n,n,n,n,n,n,NaN,NaN,NaN,NaN,NaN,91.0,31.3,34.5,12.7,177.0,93.0,NaN,46.0,88.0,2.4,118.0,0.28,cerebral_elderly_stroke
12,s0151,v1,yes,control,controlhtn,71,1.7526,99.790321,32.487987,m,nonh/l,white,nondm,yes,NaN,72.0,36.0,yes,5.0,no,no,no,no,no,no,2,0,0,0,0,no,no,no,no,no,no,nonstroke,n,n,n,n,n,n,n,n,y,n,n,n,n,6.5,5.46,16.9,nonanemic,50.7,93.0,30.9,33.3,12.3,164.0,NaN,NaN,NaN,49.0,4.5,NaN,3.76,cerebral_elderly_stroke
156,s0396,v1,yes,stroke,stroke,72,1.7200,76.000000,25.689562,m,non h/l,white,stroke,yes,no,80.0,20.0,yes,7.0,no,no,yes,no,yes,yes,2,2,0,0,0,no,yes,no,no,yes,no,stroke,y,n,y,n,n,n,NaN,n,y,n,y,n,n,7.3,4.85,14.8,nonanemic,44.3,92.0,30.6,33.4,13.3,183.0,64.0,175.0,136.0,54.0,3.2,94.0,3.80,cerebral_elderly_stroke


In [467]:
df_stroke_v2.shape

(168, 68)

## Merging and second homogenization

Before merging, let's drop the common patient so it doesn't create duplicates + this patient presents a lot of missing value

In [468]:
sub_common_perfusion_v2 = df_perfusion_v2[common_col].copy()
sub_common_stroke_v2 = df_stroke_v2[common_col].copy()
merged_common = pd.concat([sub_common_perfusion_v2, sub_common_stroke_v2])

In [469]:
merged_common.shape

(256, 56)

In [470]:
sub_common_perfusion_v2.shape

(88, 56)

In [471]:
sub_common_stroke_v2.shape

(168, 56)

In [472]:
# Checking if there is no additional missing values du to merging
check_merging(sub_common_stroke_v2, sub_common_perfusion_v2, merged_common, common_col)

ok


In [473]:
perfusion_undetected = [el[0] for el in common_col_undetected]
perfusion_undetected.append('patient id')
stroke_undetected = [el[1] for el in common_col_undetected]
stroke_undetected.append('patient id')
print(perfusion_undetected)
print(stroke_undetected)

['height (m)', 'mass (kg)', 'packyears', 'sex', 'patient id']
['height/m', 'mass/kg', 'pack years', 'gender', 'patient id']


In [474]:
sub_undetected_perfusion = df_perfusion_v2[perfusion_undetected].copy()
sub_undetected_stroke = df_stroke_v2[stroke_undetected].copy()
sub_undetected_perfusion = sub_undetected_perfusion.rename(mapper=dict(common_col_undetected), axis='columns')
print(sub_undetected_perfusion.columns)
print(sub_undetected_stroke.columns)

Index(['height/m', 'mass/kg', 'pack years', 'gender', 'patient id'], dtype='object')
Index(['height/m', 'mass/kg', 'pack years', 'gender', 'patient id'], dtype='object')


In [475]:
merged_undetected = pd.concat([sub_undetected_perfusion, sub_undetected_stroke])
merged_undetected.head()

,height/m,mass/kg,pack years,gender,patient id
0,1.790,114.65,2.86,male,s0250
1,1.531,56.90,0.14,female,s0254
2,1.520,65.77,0.00,female,s0255
3,1.575,73.50,5.71,female,s0256
4,1.700,70.31,0.00,male,s0257


In [476]:
# Checking if there is no additional missing values du to merging 
check_merging(sub_undetected_stroke,sub_undetected_perfusion, merged_undetected, stroke_undetected )

ok


From the uncommon one, the following features are kept: 
- stroke
    - completed_visit_status
    - stroke group

In [477]:
print(merged_common.shape)
print(merged_undetected.shape)
merged_v1 = pd.merge(merged_common, merged_undetected, on='patient id', how='outer')
print(merged_v1.shape)

(256, 56)
(256, 5)
(258, 60)


In [478]:
merged_v1 = pd.merge(merged_v1, df_stroke_v2[['patient id', "completed visit status"]], on='patient id', how='outer')
merged_v1 = pd.merge(merged_v1, df_stroke_v2[['patient id', "stroke group"]], on='patient id', how='outer')

In [479]:
list(merged_v1.columns)

['dm family history',
 'heart failure =chf /ifarction=mi patient medical history',
 'hgb g/dl',
 'alcohol dose/week',
 'wbc k/ul',
 'estrogen',
 'mchc %',
 'dm nondm stroke',
 'insulin(yes or no)',
 'patient id',
 'heartdisease family history',
 'ace inhibitors',
 'strokefamily history',
 'dm patient medical history',
 'stroke patient medical history',
 'cancer patient medical history',
 'atrial fibtrillation patient medical history',
 'antiparkinsonian',
 'years',
 'previous tobacco use',
 'mch pg',
 'bmi',
 'hct %',
 'statins',
 'syncope autonomic symptoms',
 'glucose mg/dl',
 'hgb<12.5',
 'anticoagulants',
 'hyperlipidemia patient medical history',
 'chol/hd ratio',
 'neuropathy autonomic symptoms',
 'study',
 'oral hypoglycemic',
 'age',
 'ethnicity',
 'current tobacco use',
 'antiplatelets',
 'previous alcohol use',
 'cancer# family history',
 'beta blockers',
 'dizziness autonomic symptoms',
 'diuretics',
 'ca ++ blockers',
 'group',
 'oh autonomic symptoms',
 'mcv fl',
 'triglyc

In [480]:
merged_v1.shape

(258, 62)

## Homogenization of str columns

In [481]:
get_unique_value_col(merged_v1, string=True)

{'0',
 '1',
 '2',
 '20',
 '3',
 '4',
 '5',
 'anemic',
 'cerebral_elderly_stroke',
 'cerebral_perfusion_diabetes',
 'ch',
 'completed',
 'control',
 'dm',
 'excluded',
 'excluded pending pcp',
 'f',
 'female',
 'h/',
 'h/l',
 'ineligible',
 'lost to followup',
 'm',
 'male',
 'mi',
 'n',
 'no',
 'non h/l',
 'nonanemic',
 'nondm',
 'nonh/l',
 'nonstroke',
 'stroke',
 'tia',
 'unknown',
 'v1',
 'y',
 'ye',
 'yes'}

In [482]:
yes = re.compile("(^yes$)|(^y$)|(^ye$)")
list(filter(yes.match, get_unique_value_col(merged_v1, string=True)))

['yes', 'y', 'ye']

In [483]:
no = re.compile("(^no$)|(^n$)")
list(filter(no.match, get_unique_value_col(merged_v1, string=True)))

['n', 'no']

In [484]:
unknown = re.compile("(unknown)")
list(filter(unknown.match, get_unique_value_col(merged_v1, string=True)))

['unknown']

In [485]:
merged_v2 = merged_v1.copy()
merged_v2 = merged_v2.replace(yes, value='yes', regex=True)
merged_v2 = merged_v2.replace(no, value='no', regex=True)
merged_v2 = merged_v2.replace(unknown, value=np.nan, regex=True)

In [486]:
print(list(filter(yes.match, get_unique_value_col(merged_v2, string=True))))
print(list(filter(no.match, get_unique_value_col(merged_v2, string=True))))
print(list(filter(unknown.match, get_unique_value_col(merged_v2, string=True))))

['yes']
['no']
[]


In [487]:
get_unique_value_col(merged_v2, string=True)

{'0',
 '1',
 '2',
 '20',
 '3',
 '4',
 '5',
 'anemic',
 'cerebral_elderly_stroke',
 'cerebral_perfusion_diabetes',
 'ch',
 'completed',
 'control',
 'dm',
 'excluded',
 'excluded pending pcp',
 'f',
 'female',
 'h/',
 'h/l',
 'ineligible',
 'lost to followup',
 'm',
 'male',
 'mi',
 'no',
 'non h/l',
 'nonanemic',
 'nondm',
 'nonh/l',
 'nonstroke',
 'stroke',
 'tia',
 'v1',
 'yes'}

In [488]:
get_unique_value_col(merged_v2)

,unique_value
dm family history,"[1, 0, 3, 2, 4, nan]"
heart failure =chf /ifarction=mi patient medical history,"[no, ch, yes, nan, mi]"
hgb g/dl,"[15.0, 14.2, nan, 10.3, 13.3, 12.1, 12.2, 12.9, 12.5, 16.2, 13.6, 10.6, 13.4, 12.0, 12.6, 14.0, 13.7, 16.8, 16.0, 14.8, 15.1, 12.4, 11.4, 14.6, 15.5, 12.7, 14.1, 15.9, 14.4, 14.9, 16.4, 11.6, 15.4, 15.8, 14.5, 13.9, 14.3, 13.0, 13.8, 12.3, 13.1, 14.7, 15.2, 16.9, 11.7, 12.8, 13.2, 15.3, 13.5, 15.6, 10.8, 16.3, 15.7, 16.1, 10.0, 17.2, 16.5, 11.9, 16.6, 11.3]"
alcohol dose/week,"[7.0, 0.5, 9.0, 3.0, 0.0, nan, 0.25, 1.0, 70.0, 4.0, 14.0, 2.0, 35.0, 0.86, 21.0, 25.0, 5.0, 10.0, 15.0, 12.0, 20.0, 1.5, 2.5, 28.0, 42.0, 6.0, 10.5, 49.0, 8.0, 3.5, 24.0]"
wbc k/ul,"[6.9, 6.7, nan, 4.4, 9.1, 6.1, 9.2, 4.9, 6.6, 8.5, 8.6, 5.6, 5.5, 5.9, 9.5, 5.2, 4.7, 8.0, 9.9, 6.2, 10.6, 10.4, 5.7, 7.3, 4.0, 7.2, 8.4, 8.3, 4.2, 8.7, 7.1, 4.8, 5.8, 6.4, 4.5, 3.6, 8.2, 6.8, 3.7, 7.0, 4.6, 7.5, 5.3, 6.3, 7.8, 11.3, 4.1, 5.4, 8.9, 11.4, 8.1, 5.0, 6.5, 3.9, 6.0, 5.1, 7.7, 9.6, 9.0, 9.4, 3.8, 2.5, 8.8, 2.9, 11.8, 12.8, 7.4, 10.2, 3.2, 11.1, 10.0, 10.3, 9.7, 13.2]"
estrogen,"[no, yes, nan]"
mchc %,"[35.5, 36.5, nan, 32.8, 34.9, 33.7, 34.4, 35.4, 34.7, 33.6, 32.5, 33.0, 32.9, 32.7, 33.8, 33.9, 33.5, 34.0, 34.8, 33.1, 35.7, 34.2, 30.6, 33.2, 34.6, 32.2, 34.1, 35.2, 32.0, 34.3, 31.4, 31.6, 32.3, 31.5, 35.3, 32.4, 34.5, 35.6, 36.4, 36.6, 33.3, 35.0, 33.4, 35.8, 35.9, 32.6, 31.7, 35.1, 36.2, 36.1, 31.8]"
dm nondm stroke,"[dm, nondm, stroke]"
insulin(yes or no),"[no, yes, nan]"
patient id,"[s0250, s0254, s0255, s0256, s0257, s0262, s0264, s0267, s0270, s0271, s0273, s0274, s0279, s0281, s0282, s0283, s0287, s0288, s0290, s0292, s0294, s0296, s0300, s0301, s0302, s0303, s0304, s0306, s0308, s0310, s0312, s0314, s0315, s0316, s0317, s0318, s0320, s0323, s0326, s0327, s0328, s0330, s0333, s0339, s0342, s0349, s0360, s0365, s0366, s0368, s0372, s0375, s0381, s0382, s0386, s0390, s0391, s0392, s0393, s0395, s0398, s0403, s0405, s0406, s0407, s0408, s0409, s0410, s0411, s0416, s0417, s0420, s0421, s0422, s0423, s0424, s0426, s0427, s0430, s0431, s0432, s0433, s0434, s0435, s0437, s0438, s0441, s0443, s0011, s0030, s0033, s0044, s0064, s0067, s0068, s0078, s0098, s0121, s0132, s0147, ...]"


Previous problems detected: 
- atrial fibtrillation patient medical history:'\h' -> np.nan
- ethnicity	[h/l, non-h/l, non h/l, nan]
- hgb<12.5	[non-anemic, nan, anemic] -> yes/no
- numbness autonomic symptoms: '20' in stroke dataset -> np.nan
- gender: [male, female, f, m]
- duplicate patient (common_patient set)
    
stroke/dm related: 
- group: DM/STROKE vs control
- dm non-dm stroke	[dm, non-dm, stroke]
- stroke group	[nan, non-stroke, stroke, tia]
- stroke patient medical history: TIA in perfusion  
    
other:
- insulin(yes_or_no): no 'yes' in the stroke dataset
- heart failure =chf /ifarction=-mi patient medical history: [no, ch, yes, nan, mi]


In [489]:
typo = re.compile('(^h/$)|(^20$)')
ethinicity_his = re.compile('(non-h/l)|(non h/l)')
anemic = re.compile('anemic')
non_anemic = re.compile('non-anemic')
female = re.compile('(^female$)|(^f$)')
male = re.compile('(^male$)|(^m$)')
print(list(filter(typo.match, get_unique_value_col(merged_v2, string=True))))
print(list(filter(ethinicity_his.match, get_unique_value_col(merged_v2, string=True))))
print(list(filter(anemic.match, get_unique_value_col(merged_v2, string=True))))
print(list(filter(non_anemic.match, get_unique_value_col(merged_v2, string=True))))
print(list(filter(male.match, get_unique_value_col(merged_v2, string=True))))
print(list(filter(female.match, get_unique_value_col(merged_v2, string=True))))

['20', 'h/']
['non h/l']
['anemic']
[]
['m', 'male']
['f', 'female']


In [490]:
merged_v3 = merged_v2.copy()
merged_v3 = merged_v3.replace(typo, value=np.nan)
merged_v3 = merged_v3.replace(ethinicity_his, value='no_hl')
merged_v3 = merged_v3.replace(anemic, value='yes')
merged_v3 = merged_v3.replace(male, value='male')
merged_v3 = merged_v3.replace(female, value='female')

In [491]:
print(list(filter(typo.match, get_unique_value_col(merged_v3, string=True))))
print(list(filter(ethinicity_his.match, get_unique_value_col(merged_v3, string=True))))
print(list(filter(anemic.match, get_unique_value_col(merged_v3, string=True))))
print(list(filter(non_anemic.match, get_unique_value_col(merged_v3, string=True))))
print(list(filter(male.match, get_unique_value_col(merged_v3, string=True))))
print(list(filter(female.match, get_unique_value_col(merged_v3, string=True))))

[]
[]
[]
[]
['male']
['female']


In [492]:
print(len(pd.unique(merged_v3['patient id'])))
print((merged_v3['patient id']).shape[0])
print(len(merged_v3[merged_v3['patient id'].isin(common_patient)]))

255
258
4


Dropping commont patients to remove duplicates and because presents a lot of missing value

In [493]:
merged_v3.drop(merged_v3[merged_v3['patient id'].isin(common_patient)].index, inplace=True)
merged_v3.shape

(254, 62)

In [494]:
stroke_dm_related_col = ['group', 'dm nondm stroke',  'stroke group', 'stroke patient medical history',  'dm patient medical history']
for col in stroke_dm_related_col:
    stroke_unique = pd.unique(merged_v3[merged_v3["study"]=='cerebral_elderly_stroke'][col])
    perfusion_unique = pd.unique(merged_v3[merged_v3["study"]=='cerebral_perfusion_diabetes'][col])
    print(f'{col} - {stroke_unique}')
    print(f'{col} - {perfusion_unique}')

group - ['control' 'stroke']
group - ['dm' 'control']
dm nondm stroke - ['nondm' 'stroke']
dm nondm stroke - ['dm' 'nondm']
stroke group - ['nonstroke' 'stroke' 'tia' nan]
stroke group - [nan]
stroke patient medical history - ['no' 'yes' nan]
stroke patient medical history - ['no' 'tia']
dm patient medical history - ['no' 'yes' nan]
dm patient medical history - ['yes' 'no']


let's check consistency between variables
- stroke
- diabetes

#### Diabetes

In [495]:
dm = re.compile('dm')
list(filter(dm.match, merged_v3.columns))

['dm family history', 'dm nondm stroke', 'dm patient medical history']

In [496]:
print(len(merged_v3[(merged_v3["group"]=='dm')]))
print(len(merged_v3[(merged_v3["dm nondm stroke"]=='dm')]))
print(len(merged_v3[(merged_v3["dm patient medical history"]=='yes')]))

print(len(merged_v3[(merged_v3["group"]=='dm') |
                    (merged_v3["dm nondm stroke"]=='dm') |
                    (merged_v3["dm patient medical history"]=='yes')]))

df_desc_v1[df_desc_v1["patient id"].isin(["dm patient medical history", 
                                              'group', 
                                              "dm, nondm stroke"])]

75
75
78
79


,patient id,s0328,s0342,variable description,units
12,group,control,dm,group ( contorl/diabetes),NaN
44,dm patient medical history,no,yes,patient with diabetes history y/n,NaN


Unconsistent number of diabetic patients

In [497]:
df_desc_v1[df_desc_v1['patient id'].str.contains(dm, regex=True, na=False)]

,patient id,s0328,s0342,variable description,units
13,dm nondm,nondm,dm,dmdiabetes mellitus,NaN
33,dm family history,1,0,diabetes mellitus family history,NaN
34,dmspecific family history,si,NaN,gp=grandparenets s = sister b= brother etc..,NaN
44,dm patient medical history,no,yes,patient with diabetes history y/n,NaN


Maybe dm, non-dm is equivalent to dm, non-dm stroke

In [498]:
print(len(merged_v3[(merged_v3["group"]=='dm') &
                    (merged_v3["dm nondm stroke"]=='dm')])) 

75


Same patients between those two features. This is because, they are all from the perfusion dataset but there is one patient noted 'dm' but no 'yes' -> drop this patient


In [499]:
merged_v3[(merged_v3["dm nondm stroke"] =='dm') &
                    (merged_v3["dm patient medical history"] !='yes')]["dm patient medical history"]

87    no
Name: dm patient medical history, dtype: object

In [500]:
merged_v4 = merged_v3.drop((merged_v3[(merged_v3["dm nondm stroke"] =='dm') &
                    (merged_v3["dm patient medical history"] !='yes')]).index)

In [501]:
# checking
merged_v4[(merged_v4["dm nondm stroke"] =='dm') &
                    (merged_v4["dm patient medical history"] !='yes')]

,dm family history,heart failure =chf /ifarction=mi patient medical history,hgb g/dl,alcohol dose/week,wbc k/ul,estrogen,mchc %,dm nondm stroke,insulin(yes or no),patient id,heartdisease family history,ace inhibitors,strokefamily history,dm patient medical history,stroke patient medical history,cancer patient medical history,atrial fibtrillation patient medical history,antiparkinsonian,years,previous tobacco use,mch pg,bmi,hct %,statins,syncope autonomic symptoms,glucose mg/dl,hgb<12.5,anticoagulants,hyperlipidemia patient medical history,chol/hd ratio,neuropathy autonomic symptoms,study,oral hypoglycemic,age,ethnicity,current tobacco use,antiplatelets,previous alcohol use,cancer# family history,beta blockers,dizziness autonomic symptoms,diuretics,ca ++ blockers,group,oh autonomic symptoms,mcv fl,triglycmg/dl,hdl mg/dl,cholestmg/dl,numbness autonomic symptoms,plt ct k/ul,arbs,antihyperlipidemic,htn family history,painful feet autonomic symptoms,crp (mg/l),height/m,mass/kg,pack years,gender,completed visit status,stroke group


In [502]:
print((merged_v4[(merged_v4["dm patient medical history"] == 'yes') &
           (merged_v4['study']=='cerebral_elderly_stroke')]).shape)

print((merged_v4[(merged_v4["dm patient medical history"] == 'yes') &
           (merged_v4['study']=='cerebral_perfusion_diabetes')]).shape)

# Explains where the four other patients come from

(4, 62)
(74, 62)


In [503]:
# checking
print(len(merged_v4[(merged_v4["group"]=='dm') |
                    (merged_v4["dm nondm stroke"]=='dm') |
                    (merged_v4["dm patient medical history"]=='yes')]))

78


-> keep "dm patient medical history" as DM: yes/no

#### stroke

stroke medical history 

Only patients from the stroke dataset have the value 'stroke'

In [504]:
print(len(merged_v4[(merged_v4["group"]=='stroke')]))
print(len(merged_v4[(merged_v4["dm nondm stroke"]=='stroke')]))
print(len(merged_v4[(merged_v4["group"]=='stroke') & (merged_v4["dm nondm stroke"]=='stroke')]))
# patients have inconsistent value

print(f'\n{merged_v4.shape}')
(merged_v4[(merged_v4["group"]=='stroke') & 
           (merged_v4["dm nondm stroke"]!='stroke')]).isna().sum(axis=1)
# on the 62 common features, those two patients have nan values for 51/50 of them -> drop

84
82
82

(253, 62)


230    51
257    50
dtype: int64

In [505]:
merged_v5 = merged_v4.drop((merged_v4[(merged_v4["group"]=='stroke') & 
                                      (merged_v4["dm nondm stroke"]!='stroke')]).index, 
                           axis=0) 
merged_v5.shape

(251, 62)

In [506]:
print(len(merged_v5[(merged_v5["stroke group"]=='stroke')]))
print(len(merged_v5[(merged_v5["dm nondm stroke"]=='stroke')]))
print(len(merged_v5[(merged_v5["group"]=='stroke')]))

76
82
82


In [507]:
(merged_v5[
    (merged_v5["group"] == 'stroke') &
    (merged_v5["dm nondm stroke"] =='stroke')]).shape

# same patients between the two, 

(82, 62)

In [508]:
(merged_v5[
    (merged_v5["stroke group"] == 'stroke') |
    (merged_v5["dm nondm stroke"] =='stroke')]).shape
# stroke group patients are included in the 82 patients found in other column

(82, 62)

In [509]:
pd.unique((merged_v5[
    (merged_v5["stroke group"] != 'stroke') &
    (merged_v5["stroke group"] != 'tia') &
    (merged_v5["dm nondm stroke"] =='stroke')])['stroke group'])

array([nan], dtype=object)

In [510]:
(merged_v5[
    (merged_v5["stroke group"] != 'stroke') &
    (merged_v5["stroke group"] != 'tia') &
    (merged_v5["dm nondm stroke"] =='stroke')]).isna().sum(axis=1)

242    51
243     6
255    15
dtype: int64

We can drop the one with 51 missing value:

In [511]:
patient_na_index = (merged_v5[
    (merged_v5["stroke group"] != 'stroke') &
    (merged_v5["stroke group"] != 'tia') &
    (merged_v5["dm nondm stroke"] =='stroke')]).isna().sum(axis=1) > 50
patient_na_index[patient_na_index==True]

242    True
dtype: bool

In [512]:
merged_v6 = merged_v5.drop(index=patient_na_index[patient_na_index==True].index)
(merged_v6[
    (merged_v6["stroke group"] != 'stroke') &
    (merged_v6["stroke group"] != 'tia') &
    (merged_v6["dm nondm stroke"] =='stroke')]).isna().sum(axis=1)

243     6
255    15
dtype: int64

In [513]:
print(merged_v5.shape)
print(merged_v6.shape)

(251, 62)
(250, 62)


## Statistics

In [514]:
print(f"Patients with dm medical history {(merged_v6['dm patient medical history']=='yes').sum()}")
print(f"Patients within the stroke group {(merged_v6['group'] == 'stroke').sum()}")
print(f"Patients within the tia in the stroke group {(merged_v6['stroke group'] == 'tia').sum()}")
print(f"Patients within the tia in the stroke medical history {(merged_v6['stroke patient medical history'] == 'tia').sum()}")
print(f"Patients within the tia in the stroke and dm {((merged_v6['group'] == 'stroke')& (merged_v6['dm patient medical history'] == 'yes')).sum()}")

Patients with dm medical history 78
Patients within the stroke group 81
Patients within the tia in the stroke group 3
Patients within the tia in the stroke medical history 1
Patients within the tia in the stroke and dm 3


Checking there are only 3 patients with a stroke and with DM

In [515]:
df_perfusion_homogenized.rename(mapper={'dm, non-dm, stroke': 'dm non-dm stroke'}, axis=1, inplace=True)
df_stroke_homogenized.rename(mapper={'dm non-dm stroke': 'dm non-dm stroke'}, axis=1, inplace=True)

In [516]:
stroke_dm_related_col = ['group', 'dm nondm stroke', 'stroke patient medical history',  'dm patient medical history']
for col in stroke_dm_related_col:
    stroke_unique = pd.unique(df_stroke_homogenized[col])
    perfusion_unique = pd.unique(df_perfusion_homogenized[col])
    print(f'{col} - {stroke_unique}')
    print(f'{col} - {perfusion_unique}')
    
print(f"stroke group - {pd.unique(df_stroke_homogenized['stroke group'])}")

group - ['control' 'stroke']
group - ['dm' 'control']
dm nondm stroke - ['nondm' 'stroke']
dm nondm stroke - ['dm' 'nondm']
stroke patient medical history - ['no' 'yes' nan]
stroke patient medical history - ['no' 'tia']
dm patient medical history - ['no' 'yes' nan]
dm patient medical history - ['yes' 'no']
stroke group - ['nonstroke' 'stroke' 'tia' nan]


In [517]:
print(f"Patients with dm medical history {(df_stroke_homogenized['dm patient medical history']=='yes').sum()}")
print(f"Patients within the stroke group {(df_stroke_homogenized['group'] == 'stroke').sum()}")
print(f"Patients with stroke and dm {((df_stroke_homogenized['dm nondm stroke'] == 'stroke') & (df_stroke_homogenized['dm patient medical history'] == 'yes')).sum()}")

Patients with dm medical history 4
Patients within the stroke group 84
Patients with stroke and dm 3


In [518]:
print(f"Patients with dm medical history {(df_perfusion_homogenized['dm patient medical history']=='yes').sum()}")
print(f"Patients within the stroke group {(df_perfusion_homogenized['stroke patient medical history'] == 'tia').sum()}")
print(f"Patients with stroke and dm {((df_perfusion_homogenized['dm nondm stroke'] == 'stroke') & (df_perfusion_homogenized['dm patient medical history'] == 'yes')).sum()}")

Patients with dm medical history 74
Patients within the stroke group 1
Patients with stroke and dm 0


Confirmation there are only 3 patients with dm and in the stroke group

In [519]:
get_plot_na(merged_v6, 1)

In [520]:
missing_values_counts_row = merged_v6.isna().sum(axis=1).sort_values(ascending=False)
missing_values_counts_row.shape #checking

(250,)

In [521]:
fig = px.bar(missing_values_counts_row.value_counts().sort_index(),height=600, width=2000, labels={'index': 'number of missing value', 'value':'number of patient'})
fig.update_layout(font_size=8, bargap=0.3)
fig.show()

In [522]:
pd.unique(merged_v6['insulin(yes or no)'])

array(['no', 'yes', nan], dtype=object)

## Listing 
- blood test measures
- medication
- cognitive tests

- crp (mg/l) ok
-cholesterol (mg/dl) ok
- hgb (g/dl)
- hgb < 12.5
- triglycemia mg/dl ok
- plt ct k/ul ok
- mch pg
- glucose mg/dl ok
- wbc k/ul
- mcv fl
- hdl mg/dl ok
- chol/hd ratio ok

In [523]:
df_desc_v1[df_desc_v1["variable description"] == "medication classess"]

,patient id,s0328,s0342,variable description,units
46,antiplatelets,n,n,medication classess,NaN
47,anticoagulants,n,n,medication classess,NaN
48,antihyperlipidemic,n,n,medication classess,NaN
49,antiparkinsonian,n,n,medication classess,NaN
50,statins,n,n,medication classess,NaN
51,estrogen,n,n,medication classess,NaN
52,ace inhibitors,n,y,medication classess,NaN
53,arbs,n,n,medication classess,NaN
54,beta blockers,n,n,medication classess,NaN
55,diuretics,n,n,medication classess,NaN


Cognitive tests features have been removed 

In [524]:
df_desc_v1.head()

,patient id,s0328,s0342,variable description,units
0,NaN,NaN,NaN,NaN,NaN
1,study,ge75,ge75,study number,NaN
2,completed visits status,completed,completed,completed study status,NaN
3,htn patient medical history,no,no,hypertension patient history,NaN
4,group 2,control,dm,group control diabetes,NaN


In [525]:
len(set(df_stroke_homogenized.columns) & set(df_perfusion_homogenized.columns))

143

In [526]:
cognitive_test = re.compile("(scale)|(test-)|(cognitive)|(\(.+\)$)")
cognitive_test_desc_ = set(filter(cognitive_test.search, pd.unique(df_desc_v1["variable description"].dropna())))
cognitive_test_var = set(filter(cognitive_test.search, pd.unique(df_desc_v1["patient id"].dropna())))
cognitive_test_stroke = set(filter(cognitive_test.search, pd.unique(df_stroke_homogenized.columns)))
cognitive_test_perfusion = set(filter(cognitive_test.search, pd.unique(df_perfusion_homogenized.columns)))

In [547]:
cognitive_test_desc

{'albumin (blood)',
 'albumin (urine)',
 'cognitive handedness (r/l)',
 'creatinine (blood)',
 'creatinine (urine)',
 'diabetic retinopathy (more advanced eye)',
 'ethnicity nonhispanic/latino ( nonh/l) hispaic latino ( h/l)',
 'gait  walk distance (m)',
 'gait  walk speed (m/s)',
 'geriatric depression scale (gds)',
 'group ( contorl/diabetes)',
 'instrumental activities of daily living scale (iadl)',
 'macular edema (more advanced eye)',
 'minimental state examination (mmse)',
 'neuropsychological impairment scale (nis)',
 'retinopathy (more advanced eye)',
 'reyosterreith complex figure (rocf)',
 'the trail making test (parts a and b)',
 'urea (blood)',
 'wechsler test of adult reading (wtar)',
 'years of education (#)'}

In [527]:
cognitive_test_desc & cognitive_test_stroke

{'cognitive handedness (r/l)'}

In [528]:
cognitive_test_var & cognitive_test_stroke

{'cognitive handedness (r/l)',
 'crp (mg/l)',
 'ds backward (item number)',
 'ds forward (items correct)',
 'ds scaled score',
 'il6 (pg/ml)',
 'insulin(yes or no)',
 'rcft copy time (sec)',
 'rcft i.r. time (sec)',
 'sicam (ng/ml)',
 'svcam (ng/ml)',
 'tm part a time (sec)',
 'tm part b time (sec)',
 'tm scaled score',
 'tnfa (pg/ml)',
 'vf # animals scaledscore',
 'vf scaled score'}

In [529]:
cognitive_test_desc & cognitive_test_perfusion

{'cognitive handedness (r/l)',
 'diabetic retinopathy (more advanced eye)',
 'macular edema (more advanced eye)'}

In [530]:
cognitive_test_var & cognitive_test_perfusion

{'cognitive handedness (r/l)',
 'crp (mg/l)',
 'diabetic retinopathy (more advanced eye)',
 'ds backward (item number)',
 'ds forward (items correct)',
 'ds scaled score',
 'eye scan (y/n)',
 'global intracranial vol (cm^3)',
 'global intracranial vol (unit ml)',
 'height (m)',
 'il6 (pg/ml)',
 'insulin(yes or no)',
 'l ventricle (#1)',
 'macular edema (more advanced eye)',
 'mass (kg)',
 'perfusion vascular (aca perf)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (acha)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto aca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto mca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto pca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (mca perf)  baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (poca)  baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (whole brain)  baseline 1 left brain (unit ml/100g/min)'

Looking of var names in the description file is more effective

In [531]:
print(len(cognitive_test_perfusion & cognitive_test_stroke))
print(len(cognitive_test_perfusion & cognitive_test_stroke & cognitive_test_desc_name))

20
17


Some variables are described in the description file or has a different name

In [532]:
cognitive_test_perfusion.difference(cognitive_test_stroke)

{'alb/cre mg/g (urine)',
 'diabetic retinopathy (more advanced eye)',
 'eye scan (y/n)',
 'global intracranial vol (cm^3)',
 'global intracranial vol (unit ml)',
 'height (m)',
 'insulin (uiu/ml)',
 'l ventricle (#1)',
 'macular edema (more advanced eye)',
 'mass (kg)',
 'perfusion vascular (aca perf)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (acha)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto aca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto mca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto pca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (mca perf)  baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (poca)  baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (whole brain)  baseline 1 left brain (unit ml/100g/min)',
 'r ventricle (#2)'}

In [533]:
cognitive_test_stroke.difference(cognitive_test_perfusion)

{'average period(sec)',
 'erythropoietin(epo)',
 'gait  walk 2 distance (m)',
 'gait  walk 2 speed (m/s)',
 'mmpf base mean phaseshift (left)',
 'mmpf base mean phaseshift (right)',
 'mmpf base std phaseshift (left)',
 'mmpf base std phaseshift (right)',
 'mmpf tilt mean phaseshift(left)',
 'mmpf tilt mean phaseshift(right)',
 'mmpf tilt std phaseshift(left)',
 'mmpf tilt std phaseshift(right)',
 'sleep(apnea%)',
 'sleep(central apnea%)',
 'sleep(data reliability%)',
 'sleep(disturbed%)',
 'sleep(restful%)',
 'sleep(stability results)',
 'sleep(wake/rem%)'}

No undetected shared elements

In [ ]:
shared_cognitives_tests = [
]

In [535]:
cognitive_test_stroke

{'average period(sec)',
 'cognitive handedness (r/l)',
 'crp (mg/l)',
 'ds backward (item number)',
 'ds forward (items correct)',
 'ds scaled score',
 'erythropoietin(epo)',
 'gait  walk 1 distance (m)',
 'gait  walk 1 speed (m/s)',
 'gait  walk 2 distance (m)',
 'gait  walk 2 speed (m/s)',
 'il6 (pg/ml)',
 'insulin(yes or no)',
 'mmpf base mean phaseshift (left)',
 'mmpf base mean phaseshift (right)',
 'mmpf base std phaseshift (left)',
 'mmpf base std phaseshift (right)',
 'mmpf tilt mean phaseshift(left)',
 'mmpf tilt mean phaseshift(right)',
 'mmpf tilt std phaseshift(left)',
 'mmpf tilt std phaseshift(right)',
 'rcft copy time (sec)',
 'rcft d.r. time (sec)',
 'rcft i.r. time (sec)',
 'sicam (ng/ml)',
 'sleep(apnea%)',
 'sleep(central apnea%)',
 'sleep(data reliability%)',
 'sleep(disturbed%)',
 'sleep(restful%)',
 'sleep(stability results)',
 'sleep(wake/rem%)',
 'svcam (ng/ml)',
 'tm part a time (sec)',
 'tm part b time (sec)',
 'tm scaled score',
 'tnfa (pg/ml)',
 'vf # animal

In [536]:
cognitive_test_stroke=[
 'average period(sec)',
 'cognitive handedness (r/l)',
 'ds backward (item number)',
 'ds forward (items correct)',
 'ds scaled score',
 'rcft copy time (sec)',
 'rcft d.r. time (sec)',
 'rcft i.r. time (sec)',
 'tm part a time (sec)',
 'tm part b time (sec)',
 'tm scaled score',
 'vf # animals scaledscore',
 'vf scaled score'
]

In [537]:
cognitive_test_perfusion

{'alb/cre mg/g (urine)',
 'cognitive handedness (r/l)',
 'crp (mg/l)',
 'diabetic retinopathy (more advanced eye)',
 'ds backward (item number)',
 'ds forward (items correct)',
 'ds scaled score',
 'eye scan (y/n)',
 'gait  walk 1 distance (m)',
 'gait  walk 1 speed (m/s)',
 'global intracranial vol (cm^3)',
 'global intracranial vol (unit ml)',
 'height (m)',
 'il6 (pg/ml)',
 'insulin (uiu/ml)',
 'insulin(yes or no)',
 'l ventricle (#1)',
 'macular edema (more advanced eye)',
 'mass (kg)',
 'perfusion vascular (aca perf)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (acha)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto aca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto mca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (lepto pca)   baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (mca perf)  baseline 1 left brain (unit ml/100g/min)',
 'perfusion vascular (poca)  baseline 1 left

In [538]:
cognitive_test_perfusion = [
 'cognitive handedness (r/l)',
 'ds backward (item number)',
 'ds forward (items correct)',
 'ds scaled score',
 'rcft copy time (sec)',
 'rcft d.r. time (sec)',
 'rcft i.r. time (sec)',
 'tm part a time (sec)',
 'tm part b time (sec)',
 'tm scaled score',
 'vf # animals scaledscore',
 'vf scaled score'
]

In [558]:
cognitive_test_shared = list(set(cognitive_test_perfusion) & set(cognitive_test_stroke))

['ds backward (item number)',
 'rcft d.r. time (sec)',
 'rcft i.r. time (sec)',
 'ds forward (items correct)',
 'tm part a time (sec)',
 'tm scaled score',
 'vf scaled score',
 'rcft copy time (sec)',
 'ds scaled score',
 'vf # animals scaledscore',
 'cognitive handedness (r/l)',
 'tm part b time (sec)']

In [539]:
print(len(cognitive_test_perfusion))
print(len(cognitive_test_stroke))

12
13


In [540]:
pd.merge(df_desc_v1[(df_desc_v1['patient id'].isin(cognitive_test_perfusion))],
         pd.Series(cognitive_test_perfusion, name="patient id"), 
         on='patient id', 
         how='outer')

,patient id,s0328,s0342,variable description,units
0,cognitive handedness (r/l),right,right,cognitive handedness (r/l),right/left
1,ds forward (items correct),11,11,digit span test,NaN
2,ds backward (item number),2,4,digit span test,NaN
3,ds scaled score,9,12,digit span test,NaN
4,rcft copy time (sec),126,211,reyosterreith complex figure (rocf),sec
5,rcft i.r. time (sec),70,83,reyosterreith complex figure (rocf),sec
6,vf scaled score,13,9,verbal fluency test,NaN
7,vf # animals scaledscore,10,8,verbal fluency test,NaN
8,tm part a time (sec),28,63,the trail making test (parts a and b),sec
9,tm part b time (sec),70,120,the trail making test (parts a and b),sec


In [541]:
pd.merge(df_desc_v1[(df_desc_v1['patient id'].isin(cognitive_test_stroke))],
         pd.Series(cognitive_test_stroke, name="patient id"), 
         on='patient id', 
         how='outer')

,patient id,s0328,s0342,variable description,units
0,cognitive handedness (r/l),right,right,cognitive handedness (r/l),right/left
1,ds forward (items correct),11,11,digit span test,NaN
2,ds backward (item number),2,4,digit span test,NaN
3,ds scaled score,9,12,digit span test,NaN
4,rcft copy time (sec),126,211,reyosterreith complex figure (rocf),sec
5,rcft i.r. time (sec),70,83,reyosterreith complex figure (rocf),sec
6,vf scaled score,13,9,verbal fluency test,NaN
7,vf # animals scaledscore,10,8,verbal fluency test,NaN
8,tm part a time (sec),28,63,the trail making test (parts a and b),sec
9,tm part b time (sec),70,120,the trail making test (parts a and b),sec


In [559]:
pd.merge(df_desc_v1[df_desc_v1['patient id'].isin(cognitive_test_shared)],
         pd.Series(cognitive_test_shared, name="patient id"), 
         on='patient id', 
         how='outer')

,patient id,s0328,s0342,variable description,units
0,cognitive handedness (r/l),right,right,cognitive handedness (r/l),right/left
1,ds forward (items correct),11,11,digit span test,NaN
2,ds backward (item number),2,4,digit span test,NaN
3,ds scaled score,9,12,digit span test,NaN
4,rcft copy time (sec),126,211,reyosterreith complex figure (rocf),sec
5,rcft i.r. time (sec),70,83,reyosterreith complex figure (rocf),sec
6,vf scaled score,13,9,verbal fluency test,NaN
7,vf # animals scaledscore,10,8,verbal fluency test,NaN
8,tm part a time (sec),28,63,the trail making test (parts a and b),sec
9,tm part b time (sec),70,120,the trail making test (parts a and b),sec


Drop columns unwanted columns linked to biomeasures

In [543]:
bio_measures_drop = ['wbc k/ul', 'mcv fl', 'mch pg', 'hgb g/dl', 'hgb<12.5']
merged_v7 = merged_v6.drop(bio_measures_drop, axis='columns')
print(merged_v6.shape)
print(merged_v7.shape)
print(len(bio_measures_drop))

(250, 62)
(250, 57)
5


In [544]:
missing_values_counts = merged_v7.isna().sum().sort_values(ascending=False)
missing_values_counts.shape #checking

(57,)

In [545]:
get_plot_na(merged_v7, 0)